Multivariate Time Series Forecasting uygulamamız için modelimizi eğitiyoruz

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

In [3]:
# Veriyi Yükleyip EDA

In [4]:
df = pd.read_csv("stocks.csv")

In [5]:
df.head()

,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
0,AAPL,2023-02-07,150.639999,155.229996,150.639999,154.649994,154.414230,83322600
1,AAPL,2023-02-08,153.880005,154.580002,151.169998,151.919998,151.688400,64120100
2,AAPL,2023-02-09,153.779999,154.330002,150.419998,150.869995,150.639999,56007100
3,AAPL,2023-02-10,149.460007,151.339996,149.220001,151.009995,151.009995,57450700
4,AAPL,2023-02-13,150.949997,154.259995,150.919998,153.850006,153.850006,62199000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Ticker     248 non-null    object 
 1   Date       248 non-null    object 
 2   Open       248 non-null    float64
 3   High       248 non-null    float64
 4   Low        248 non-null    float64
 5   Close      248 non-null    float64
 6   Adj Close  248 non-null    float64
 7   Volume     248 non-null    int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 15.6+ KB


In [7]:
df.isnull().sum()

Ticker       0
Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [8]:
# GOOG hissesini seçiyoruz

In [9]:
data = df[df['Ticker'] == 'GOOG'].copy()

In [10]:
# Tarihe göre sıralıyoruz

In [11]:
data['Date'] = pd.to_datetime(data['Date'])

In [12]:
data = data.sort_values('Date')

In [13]:
# Kullanılacak Özellikler (Multivariate)

In [14]:
features = ['Open', 'High', 'Low', 'Volume', 'Close']

In [15]:
dataset = data[features].values

In [16]:
# Ölçeklendirme (MinMax Scaling - LSTM için çok önemlidir)

In [17]:
from sklearn.preprocessing import MinMaxScaler

In [18]:
scaler = MinMaxScaler(feature_range=(0, 1))

In [19]:
scaled_data = scaler.fit_transform(dataset)

In [20]:
# Zaman Serisi Dizileri Oluşturma (Sliding Window)

In [21]:
def create_sequences(data, seq_length=60):
    X = []
    y = []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i]) # Geçmiş 60 gün (Tüm özellikler)
        y.append(data[i, 4]) # 4. indeks 'Close' sütunudur (Hedef)
    return np.array(X), np.array(y)

In [22]:
SEQ_LENGTH = 60

In [23]:
import numpy as np

In [24]:
X, y = create_sequences(scaled_data, SEQ_LENGTH)

In [25]:
# Veriyi Train/Test olarak ayırma (%80 Train)

In [26]:
train_size = int(len(X) * 0.8)

In [27]:
X_train, X_test = X[:train_size], X[train_size:]

In [28]:
y_train, y_test = y[:train_size], y[train_size:]

In [29]:
# LSTM Model Mimarisi

In [30]:
from tensorflow.keras.models import Sequential

In [31]:
model = Sequential()

In [32]:
# 1. LSTM Katmanı

In [33]:
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [34]:
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))

In [35]:
model.add(Dropout(0.2))

In [36]:
# 2. LSTM Katmanı

In [37]:
model.add(LSTM(units=50, return_sequences=False))

In [38]:
model.add(Dropout(0.2))

In [39]:
# Çıktı Katmanı

In [40]:
model.add(Dense(units=1)) # Tek bir değer tahmin ediyoruz (Close Price)

In [41]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [42]:
# Modeli Eğitme

In [43]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.2708 - val_loss: 0.2379
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.1741 - val_loss: 0.1153
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.0963 - val_loss: 0.0323
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.0024 - val_loss: 0.0031
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - loss: 0.0049 - val_loss: 8.9269e-04
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 0.0466 - val_loss: 0.0025
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 0.0161 - val_loss: 0.0018
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 0.0096 - val_loss: 3.4890e-04
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 0.0021 - val_loss: 1.8321e-05
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.0123 - val_loss: 3.6643e-05
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 9.0316e-06 - val_loss: 4.6153e-05
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.00

In [44]:
# Save

In [45]:
model.save('stock_lstm_model.h5')

In [46]:
import joblib

In [47]:
joblib.dump(scaler, 'stock_scaler.pkl')

['stock_scaler.pkl']